<a href="https://colab.research.google.com/github/Djadih/DataScience_Project/blob/main/midi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
uploaded = files.upload()

Saving mz_331_1.mid to mz_331_1.mid


In [ ]:
! pip install pygame

     |████████████████████████████████| 11.8MB 297kB/s 


In [7]:
import string
import numpy as np
from music21 import converter, instrument, note, chord, stream, tempo, meter

In [ ]:
midi = converter.parse('mz_331_1.mid')

In [267]:
max = 30
j = 0
ins = instrument.partitionByInstrument(midi)
notes_to_parse = ins.parts[0].recurse() 
for i in notes_to_parse:
  print(i, i.offset)
  j+=1
  if j ==max:
    break

Piano 0.0
<music21.tempo.MetronomeMark Quarter=63.51> 0.0
A major 0.0
A major 0.0
<music21.meter.TimeSignature 6/8> 0.0
<music21.meter.TimeSignature 6/8> 0.0
<music21.note.Note C#> 0.0
<music21.note.Rest rest> 0.0
<music21.note.Note A> 0.0
<music21.note.Note E> 0.0
<music21.tempo.MetronomeMark andante Quarter=70.39> 0.75
<music21.note.Note D> 0.75
<music21.note.Note B> 0.75
<music21.tempo.MetronomeMark larghetto Quarter=58.91> 1.0
<music21.note.Note C#> 1.0
<music21.chord.Chord A3 E4> 1.0
<music21.tempo.MetronomeMark Quarter=63.51> 1.5
<music21.note.Note E> 1.5
<music21.chord.Chord E4 C#4> 1.5
<music21.tempo.MetronomeMark adagio Quarter=55.85> 2.5
<music21.note.Note E> 2.5
<music21.chord.Chord C#4 E4> 2.5
<music21.tempo.MetronomeMark adagietto Quarter=65.03> 3.0
<music21.note.Note B> 3.0
<music21.note.Note G#> 3.0
<music21.note.Note E> 3.0
<music21.note.Rest rest> 3.0
<music21.tempo.MetronomeMark andante Quarter=70.39> 3.75
<music21.note.Note C#> 3.75
<music21.note.Note A> 3.75


In [49]:
def get_notes(filename):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    temp = 0
    temp_string = ''
    temp_found = False
    st = ''
    midi = converter.parse(filename)
    print("Parsing %s" % filename)
    notes_to_parse = None

    try: # file has instrument parts
      ins = instrument.partitionByInstrument(midi)
      notes_to_parse = ins.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, tempo.MetronomeMark) and not temp_found:
        temp = element.getQuarterBPM()
        temp_string = element.text
        temp_found = True
      elif isinstance(element, meter.TimeSignature):
        st = element.ratioString

      elif isinstance(element, note.Note):
        notes.append((element.pitch.midi, element.offset))
      elif isinstance(element, chord.Chord):
        notes.append(([n.midi for n in element.pitches], element.offset))

    return notes, temp, temp_string, st

def get_dict(filename):
    notes, temp, temp_string, st = get_notes(filename)
    music = {}
    music['tempo_num'] = temp
    music['tempo_string'] = temp_string
    music['time_signature'] = st
    return music, notes

In [50]:
song, notes = get_dict('mz_331_1.mid')

Parsing mz_331_1.mid


In [35]:
song

{'tempo_num': 63.51, 'tempo_string': None, 'time_signature': '4/4'}

In [51]:
for i in notes[:20]:
  print(i)

(73, 0.0)
(57, 0.0)
(64, 0.0)
(74, 0.75)
(59, 0.75)
(73, 1.0)
([57, 64], 1.0)
(76, 1.5)
([64, 61], 1.5)
(76, 2.5)
([61, 64], 2.5)
(71, 3.0)
(56, 3.0)
(64, 3.0)
(73, 3.75)
(57, 3.75)
(71, 4.0)
([56, 64], 4.0)
(74, 4.5)
([64, 59], 4.5)


In [18]:
def create_midi(prediction_dict, notes):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    output_notes = []
    temp_num = prediction_dict['tempo_num']
    temp_string = prediction_dict['tempo_string']
    st = prediction_dict['time_signature']

    mm1 = tempo.MetronomeMark(temp_string, temp_num)
    ts = meter.TimeSignature(st)
    output_notes.append(mm1)
    output_notes.append(ts)

    # create note and chord objects based on the values generated by the model
    for pattern, offset in notes:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(current_note)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that not  es do not stack

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output_tempo.mid')

In [20]:
create_midi(song, notes[:50])

In [22]:
test = note.Note(64)

In [31]:
test.

'E4'